In [26]:
from dotenv import load_dotenv
import os
load_dotenv(".env")

True

In [27]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_NAME"] = "zep"
os.environ["WANDB_NOTES"] = "zepmem"
os.environ["WANDB_PROJECT"] = "langchain-tracing"
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks import wandb_tracing_enabled
from langchain_openai import OpenAI
os.environ["WANDB_NOTEBOOK_NAME"]="./runnablemem.ipynb"

In [3]:
from langchain.schema import Document
from langchain_community.vectorstores import Redis
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [4]:
from typing import Optional

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_models import ChatAnthropic
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
from IPython.display import Markdown
from langchain.agents.load_tools import get_all_tool_names
from langchain import hub
prompt = hub.pull("hwchase17/react-chat")
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent, create_react_agent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import OpenAI
from langchain import hub
from langchain_openai import ChatOpenAI

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an assistant who's good at {ability}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI(model="gpt-4-1106-preview",temperature=0.4)

In [8]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: RedisChatMessageHistory(session_id, url="redis://localhost:6379/0"),
    input_messages_key="question",
    history_messages_key="history",
)

config={"configurable": {"session_id": "<SESSION_ID>"}}

In [9]:
chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

wandb: Streaming LangChain activity to W&B at https://wandb.ai/bradley-euell/langchain-tracing/runs/0fwf3ghz
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


AIMessage(content='Cosine is a trigonometric function that relates the angle of a right triangle to the ratio of the length of the adjacent side to the length of the hypotenuse. It is one of the fundamental functions in trigonometry and is abbreviated as "cos."\n\nHere\'s a more detailed explanation:\n\nIn a right-angled triangle:\n\n- The hypotenuse is the side opposite the right angle and is the longest side of the triangle.\n- The adjacent side is the side next to the angle of interest (other than the hypotenuse).\n- The opposite side is the side across from the angle of interest.\n\nThe cosine of an angle (θ) is defined as:\n\ncos(θ) = length of the adjacent side / length of the hypotenuse\n\nFor example, if you have a right-angled triangle where the length of the hypotenuse is 10 units and the length of the side adjacent to the angle θ is 8 units, then the cosine of angle θ would be:\n\ncos(θ) = 8 / 10 = 0.8\n\nCosine is also a periodic function, which means it repeats its values 

In [10]:
chain_with_history.invoke(
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content="The inverse of the cosine function is called the arccosine function, often denoted as arccos or cos^(-1). The arccosine function takes a value between -1 and 1 (the range of the cosine function) and returns an angle, typically measured in radians or degrees, whose cosine is the given value.\n\nFor example, if you have:\n\ncos(θ) = x\n\nThen the inverse is:\n\nθ = arccos(x)\n\nThe domain of the arccosine function (the set of values for which the function is defined) is [-1, 1], and its range (the set of possible output values) is typically [0, π] radians or [0°, 180°] when measured in degrees. This is because, for any given value within [-1, 1], there is a unique angle within this range that has that value as its cosine.\n\nIt's important to note that since the cosine function is not one-to-one (it does not pass the horizontal line test), its inverse is not defined for all real numbers. To define an inverse, we must restrict the domain of the cosine function to an int

In [11]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": ChatOpenAI(model="gpt-4-1106-preview",temperature=0.4)})
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: RedisChatMessageHistory(session_id, url="redis://localhost:6379/0"),
    output_messages_key="output_message",
)

chain_with_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='Simone de Beauvoir, a French existentialist philosopher, writer, and social theorist, is best known for her contributions to existentialism and feminism. Her views on free will are closely tied to her existentialist beliefs, which she shared with her lifelong partner, Jean-Paul Sartre.\n\nDe Beauvoir believed that individuals have free will and are responsible for creating meaning in their lives through their choices and actions. She argued that humans are "condemned to be free," a concept she borrowed from Sartre, which suggests that we are thrown into existence without a predetermined essence or purpose and must define ourselves through our decisions and commitments.\n\nIn her seminal work "The Ethics of Ambiguity," de Beauvoir delves into the concept of ambiguity, which refers to the tension between our freedom (the ability to transcend our given situation) and our facticity (the concrete details of our lives, including our past, our body, socia

In [12]:
chain_with_history.invoke(
    [HumanMessage(content="How did this compare to Sartre")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='Simone de Beauvoir\'s views on free will are largely consistent with those of Jean-Paul Sartre, as both were prominent existentialist thinkers who influenced each other\'s work. However, while their fundamental ideas on freedom and existentialism were aligned, there were nuances in their approaches and emphases.\n\nSartre, in his existentialist philosophy, famously declared that "existence precedes essence," meaning that individuals are not born with a predetermined nature or purpose. Instead, they are free to define themselves through their choices and actions. Sartre emphasized radical freedom and the idea that individuals are always free to choose, even in situations that seem to constrain their options. According to Sartre, this freedom is inescapable and comes with the weight of responsibility, as individuals must own up to their choices and cannot blame their circumstances or nature for their actions.\n\nDe Beauvoir shared Sartre\'s view on t

In [13]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
import redis

message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0", ttl=600, session_id="my-session"
)

memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=message_history
)

In [14]:
import time
from uuid import uuid4

from langchain.memory import ZepMemory
from langchain.schema import AIMessage, HumanMessage

In [18]:
ZEP_API_URL = os.environ["ZEP_API_URL"]
# Provide your Zep API key. Note that this is optional. See https://docs.getzep.com/deployment/auth
AUTHENTICATE = False

zep_api_key = None
if AUTHENTICATE:
    zep_api_key = os.environ["ZEP_API_KEY"]

In [22]:
session_id = str(uuid4())  # This is a unique identifier for the user/session

# Initialize the Zep Memory Class
zep_memory = ZepMemory(session_id=session_id, url=ZEP_API_URL, api_key=zep_api_key)

In [20]:
%pip install zep-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.2
    Uninstalling httpcore-1.0.2:
      Successfully uninstalled httpcore-1.0.2
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0
Note: you may need to restart the kernel to use updated packages.


In [23]:
# Preload some messages into the memory. The default message window is 12 messages. We want to push beyond this to demonstrate auto-summarization.
test_history = [
    {"role": "human", "content": "Who was Octavia Butler?"},
    {
        "role": "ai",
        "content": (
            "Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American"
            " science fiction author."
        ),
    },
    {"role": "human", "content": "Which books of hers were made into movies?"},
    {
        "role": "ai",
        "content": (
            "The most well-known adaptation of Octavia Butler's work is the FX series"
            " Kindred, based on her novel of the same name."
        ),
    },
    {"role": "human", "content": "Who were her contemporaries?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler's contemporaries included Ursula K. Le Guin, Samuel R."
            " Delany, and Joanna Russ."
        ),
    },
    {"role": "human", "content": "What awards did she win?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur"
            " Fellowship."
        ),
    },
    {
        "role": "human",
        "content": "Which other women sci-fi writers might I want to read?",
    },
    {
        "role": "ai",
        "content": "You might want to read Ursula K. Le Guin or Joanna Russ.",
    },
    {
        "role": "human",
        "content": (
            "Write a short synopsis of Butler's book, Parable of the Sower. What is it"
            " about?"
        ),
    },
    {
        "role": "ai",
        "content": (
            "Parable of the Sower is a science fiction novel by Octavia Butler,"
            " published in 1993. It follows the story of Lauren Olamina, a young woman"
            " living in a dystopian future where society has collapsed due to"
            " environmental disasters, poverty, and violence."
        ),
    },
    {"role": "human", "content": "What is the setting of the book?"},
    {
        "role": "ai",
        "content": (
            "The book is set in a dystopian future in the 2020s, where society has"
            " collapsed due to climate change and economic crises."
        ),
    },
    {"role": "human", "content": "Who is the protagonist?"},
    {
        "role": "ai",
        "content": (
            "The protagonist of the book is Lauren Olamina, a young woman who possesses"
            " 'hyperempathy', the ability to feel pain and other sensations she"
            " witnesses."
        ),
    },
    {"role": "human", "content": "What is the main theme of the book?"},
    {
        "role": "ai",
        "content": (
            "The main theme of the book is survival in the face of drastic societal"
            " change and collapse. It also explores themes of adaptability, community,"
            " and the human capacity for change."
        ),
    },
    {"role": "human", "content": "What is the 'Parable of the Sower'?"},
    {
        "role": "ai",
        "content": (
            "The 'Parable of the Sower' is a biblical parable that Butler uses as a"
            " metaphor in the book. In the parable, a sower scatters seeds, some of"
            " which fall on fertile ground and grow, while others fall on rocky ground"
            " or among thorns and fail to grow. The parable is used to illustrate the"
            " importance of receptivity and preparedness in the face of change."
        ),
    },
    {"role": "human", "content": "What is Butler's writing style like?"},
    {
        "role": "ai",
        "content": (
            "Butler's writing style is known for its clarity, directness, and"
            " psychological insight. Her narratives often involve complex, diverse"
            " characters and explore themes of race, gender, and power."
        ),
    },
    {"role": "human", "content": "What other books has she written?"},
    {
        "role": "ai",
        "content": (
            "In addition to 'Parable of the Sower', Butler has written several other"
            " notable works, including 'Kindred', 'Dawn', and 'Parable of the Talents'."
        ),
    },
]

for msg in test_history:
    zep_memory.chat_memory.add_message(
        HumanMessage(content=msg["content"])
        if msg["role"] == "human"
        else AIMessage(content=msg["content"])
    )

time.sleep(
    10
)  # Wait for the messages to be embedded and summarized. Speed depends on OpenAI API latency and your rate limits.

In [28]:
from langchain.retrievers import ZepRetriever
from langchain.retrievers.zep import SearchScope, SearchType

zep_retriever = ZepRetriever(
    session_id=session_id,  # Ensure that you provide the session_id when instantiating the Retriever
    url=ZEP_API_URL,
    top_k=5,
    api_key=zep_api_key,
)

await zep_retriever.aget_relevant_documents("Who wrote Parable of the Sower?")

[Document(page_content="What is the 'Parable of the Sower'?", metadata={'score': 0.9250596761703491, 'uuid': 'b2a4b400-633b-4a8d-a63b-4557c8928043', 'created_at': '2024-01-14T06:48:53.498734Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'human', 'metadata': {'system': {'entities': [{'Label': 'WORK_OF_ART', 'Matches': [{'End': 34, 'Start': 13, 'Text': "Parable of the Sower'"}], 'Name': "Parable of the Sower'"}], 'intent': "The subject is asking for an explanation or summary of the 'Parable of the Sower'."}}, 'token_count': 0}),
 Document(page_content='Parable of the Sower is a science fiction novel by Octavia Butler, published in 1993. It follows the story of Lauren Olamina, a young woman living in a dystopian future where society has collapsed due to environmental disasters, poverty, and violence.', metadata={'score': 0.8897120952606201, 'uuid': '2d01f292-8bc2-4cb6-b4c0-9520429ccfb4', 'created_at': '2024-01-14T06:48:53.095594Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'ai', 'm

In [29]:
zep_retriever.get_relevant_documents("Who wrote Parable of the Sower?")

[Document(page_content="What is the 'Parable of the Sower'?", metadata={'score': 0.9250596761703491, 'uuid': 'b2a4b400-633b-4a8d-a63b-4557c8928043', 'created_at': '2024-01-14T06:48:53.498734Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'human', 'metadata': {'system': {'entities': [{'Label': 'WORK_OF_ART', 'Matches': [{'End': 34, 'Start': 13, 'Text': "Parable of the Sower'"}], 'Name': "Parable of the Sower'"}], 'intent': "The subject is asking for an explanation or summary of the 'Parable of the Sower'."}}, 'token_count': 0}),
 Document(page_content='Parable of the Sower is a science fiction novel by Octavia Butler, published in 1993. It follows the story of Lauren Olamina, a young woman living in a dystopian future where society has collapsed due to environmental disasters, poverty, and violence.', metadata={'score': 0.8897120952606201, 'uuid': '2d01f292-8bc2-4cb6-b4c0-9520429ccfb4', 'created_at': '2024-01-14T06:48:53.095594Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'ai', 'm

In [30]:
zep_retriever = ZepRetriever(
    session_id=session_id,  # Ensure that you provide the session_id when instantiating the Retriever
    url=ZEP_API_URL,
    top_k=5,
    api_key=zep_api_key,
    search_type=SearchType.mmr,
    mmr_lambda=0.5,
)

await zep_retriever.aget_relevant_documents("Who wrote Parable of the Sower?")

[Document(page_content="What is the 'Parable of the Sower'?", metadata={'score': 0.9250596761703491, 'uuid': 'b2a4b400-633b-4a8d-a63b-4557c8928043', 'created_at': '2024-01-14T06:48:53.498734Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'human', 'metadata': {'system': {'entities': [{'Label': 'WORK_OF_ART', 'Matches': [{'End': 34, 'Start': 13, 'Text': "Parable of the Sower'"}], 'Name': "Parable of the Sower'"}], 'intent': "The subject is asking for an explanation or summary of the 'Parable of the Sower'."}}, 'token_count': 0}),
 Document(page_content='What other books has she written?', metadata={'score': 0.7748876810073853, 'uuid': '4d5ba350-7f9f-4b38-b989-2b983da3f456', 'created_at': '2024-01-14T06:48:53.791968Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'human', 'metadata': {'system': {'entities': [], 'intent': 'Inquiring about the other books written by a specific author.'}}, 'token_count': 0}),
 Document(page_content="In addition to 'Parable of the Sower', Butler has writte

In [31]:
filter = {"where": {"jsonpath": '$[*] ? (@.Label == "WORK_OF_ART")'}}

await zep_retriever.aget_relevant_documents(
    "Who wrote Parable of the Sower?", metadata=filter
)


[Document(page_content="What is the 'Parable of the Sower'?", metadata={'score': 0.9250596761703491, 'uuid': 'b2a4b400-633b-4a8d-a63b-4557c8928043', 'created_at': '2024-01-14T06:48:53.498734Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'human', 'metadata': {'system': {'entities': [{'Label': 'WORK_OF_ART', 'Matches': [{'End': 34, 'Start': 13, 'Text': "Parable of the Sower'"}], 'Name': "Parable of the Sower'"}], 'intent': "The subject is asking for an explanation or summary of the 'Parable of the Sower'."}}, 'token_count': 0}),
 Document(page_content='What other books has she written?', metadata={'score': 0.7748876810073853, 'uuid': '4d5ba350-7f9f-4b38-b989-2b983da3f456', 'created_at': '2024-01-14T06:48:53.791968Z', 'updated_at': '0001-01-01T00:00:00Z', 'role': 'human', 'metadata': {'system': {'entities': [], 'intent': 'Inquiring about the other books written by a specific author.'}}, 'token_count': 0}),
 Document(page_content="In addition to 'Parable of the Sower', Butler has writte

In [32]:
zep_retriever = ZepRetriever(
    session_id=session_id,  # Ensure that you provide the session_id when instantiating the Retriever
    url=ZEP_API_URL,
    top_k=3,
    api_key=zep_api_key,
    search_scope=SearchScope.summary,
    search_type=SearchType.mmr,
    mmr_lambda=0.5,
)

await zep_retriever.aget_relevant_documents("Who wrote Parable of the Sower?")

[Document(page_content='The AI provides information about Octavia Butler\'s book "Parable of the Sower," including the dystopian setting in the 2020s, the protagonist Lauren Olamina with the ability of \'hyperempathy\', and the main theme of survival and societal collapse.', metadata={'score': 0.8266599178314209, 'uuid': '5f565f47-e2c2-4172-8371-8bc6f7d7b5b2', 'created_at': '2024-01-14T06:48:58.222516Z', 'token_count': 0}),
 Document(page_content="The human asks about movies based on Octavia Butler's books, and the AI mentions the FX series Kindred. The human then asks about her contemporaries, and the AI lists Ursula K. Le Guin, Samuel R. Delany, and Joanna Russ. Finally, the human inquires about the awards Octavia Butler won, and the AI mentions the Hugo Award, the Nebula Award, and the MacArthur Fellowship.", metadata={'score': 0.7454637289047241, 'uuid': 'a3453386-7879-4e96-9c6a-26359a75010c', 'created_at': '2024-01-14T06:48:56.093355Z', 'token_count': 0})]